# Fine-Tuning with Llama 2, Bits and Bytes, and QLoRA

Today we'll explore fine-tuning the Llama 2 model available using QLoRA, Bits and Bytes, and PEFT.

- QLoRA: [Quantized Low Rank Adapters](https://arxiv.org/pdf/2305.14314.pdf) - this is a method for fine-tuning LLMs that uses a small number of quantized, updateable parameters to limit the complexity of training. This technique also allows those small sets of parameters to be added efficiently into the model itself, which means you can do fine-tuning on lots of data sets, potentially, and swap these "adapters" into your model when necessary.
- [Bits and Bytes](https://github.com/TimDettmers/bitsandbytes): An excellent package by Tim Dettmers et al., which provides a lightweight wrapper around custom CUDA functions that make LLMs go faster - optimizers, matrix mults, and quantization. In this notebook we'll be using the library to load our model as efficiently as possible.
- [PEFT](https://github.com/huggingface/peft): An excellent Huggingface library that enables a number Parameter Efficient Fine-tuning (PEFT) methods, which again make it less expensive to fine-tune LLMs - especially on more lightweight hardware like that present in Kaggle notebooks.

Many thanks to [Bojan Tunguz](https://www.kaggle.com/tunguz) for his excellent [Jeopardy dataset](https://www.kaggle.com/datasets/tunguz/200000-jeopardy-questions)!

This notebook is based on [an excellent example from LangChain](https://github.com/asokraju/LangChainDatasetForge/blob/main/Finetuning_Falcon_7b.ipynb).

## Package Installation

Note that we're loading very specific versions of these libraries. Dependencies in this space can be quite difficult to untangle, and simply taking the latest version of each library can lead to conflicting version requirements. It's a good idea to take note of which versions work for your particular use case, and `pip install` them directly.

In [ ]:
%pip install -qqq bitsandbytes
%pip install -qqq torch
%pip install -qqq -U git+https://github.com/huggingface/transformers.git
%pip install -qqq -U git+https://github.com/huggingface/peft.git
%pip install -qqq -U git+https://github.com/huggingface/accelerate.git
%pip install -qqq datasets
%pip install -qqq loralib
%pip install -qqq einops

In [ ]:
%pip install -q optimum auto-gptq pandas

In [ ]:
import pandas as pd
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Loading and preparing our model

We're going to use the Llama 2 7B model for our test. We'll be using Bits and Bytes to load it in 4-bit format, which should reduce memory consumption considerably, at a cost of some accuracy.

Note the parameters in `BitsAndBytesConfig` - this is a fairly standard 4-bit quantization configuration, loading the weights in 4-bit format, using a straightforward format (`normal float 4`) with double quantization to improve QLoRA's resolution. The weights are converted back to `bfloat16` for weight updates, then the extra precision is discarded.

In [ ]:
# model = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
# MODEL_NAME = model

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     trust_remote_code=True,
#     quantization_config=bnb_config
# )

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# model = "google/gemma-2b-it"
# MODEL_NAME = model

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     trust_remote_code=True,
#     quantization_config=bnb_config,
#     token='<token>'
# )

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token='<token>')
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Below, we'll use a nice PEFT wrapper to set up our model for training / fine-tuning. Specifically this function sets the output embedding layer to allow gradient updates, as well as performing some type casting on various components to ensure the model is ready to be updated.

In [ ]:
model = prepare_model_for_kbit_training(model)

Below, we define some helper functions - their purpose is to properly identify our update layers so we can... update them!

In [ ]:
model

In [ ]:
import re
def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    print(max(numbers))
    return max(numbers)

def get_last_layer_linears(model):
    names = []
    
    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    print(names)
    return names

## LORA config

Some key elements from this configuration:
1. `r` is the width of the small update layer. In theory, this should be set wide enough to capture the complexity of the problem you're attempting to fine-tune for. More simple problems may be able to get away with smaller `r`. In our case, we'll go very small, largely for the sake of speed.
2. `target_modules` is set using our helper functions - every layer identified by that function will be included in the PEFT update.

In [ ]:
config = LoraConfig(
    r=2,
    lora_alpha=32,
    target_modules=get_last_layer_linears(model),
    # target_modules = ["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

## Load some data

Here, we're loading a 200,000 question Jeopardy dataset. In the interests of time we won't load all of them - just the first 1000 - but we'll fine-tune our model using the question and answers. Note that what we're training the model to do is use its existing knowledge (plus whatever little it learns from our question-answer pairs) to answer questions in the *format* we want, specifically short answers.

In [ ]:
df = pd.read_csv("readme_qa.csv") # Use readme_qa.csv generated from data.ipynb
df.columns = [str(q).strip() for q in df.columns]

data = Dataset.from_pandas(df)

In [ ]:
# prompt = df["Question"].values[0] + ". Answer as briefly as possible: ".strip()
# prompt

In [ ]:
project_name = df["Repo"].values[10820]
repository_url = df["Repo Url"].values[10820]
target_audience = "smart developer"
question = df["Question"].values[10820]
context = df["Context"].values[10820]
content_type = "docs"
prompt = f"""You are an AI assistant for a software project called {project_name}. You are trained on all the {content_type} that makes up this project.
    The {content_type} for the project is located at {repository_url}.
    You are given a repository which might contain several modules and each module will contain a set of files.
    Look at the source code in the repository and you have to generate content for the section of a README.md file following the heading given below. If you use any hyperlinks, they should link back to the github repository shared with you.
    You should only use hyperlinks that are explicitly listed in the context. Do NOT make up a hyperlink that is not listed.

    Assume the reader is a {target_audience} but is not deeply familiar with {project_name}.
    Assume the reader does not know anything about how the project is structured or which folders/files do what and what functions are written in which files and what these functions do.
    If you don't know how to fill up the readme.md file in one of its sections, leave that part blank. Don't try to make up any content.
    Do not include information that is not directly relevant to repository, even though the names of the functions might be common or is frequently used in several other places.
    Keep your response between 100 and 300 words. DO NOT RETURN MORE THAN 300 WORDS. Provide the answer in correct markdown format.

    Question: {question}
    Context:
    {context}

    Answer in Markdown:"""
prompt

## Let's generate!

Below we're setting up our generative model:
- Top P: a method for choosing from among a selection of most probable outputs, as opposed to greedily just taking the highest)
- Temperature: a modulation on the softmax function used to determine the values of our outputs
- We limit the return sequences to 1 - only one answer is allowed! - and deliberately force the answer to be short.

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 512
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.repetition_penalty = 1.1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

Now, we'll generate an answer to our first question, just to see how the model does!

It's fascinatingly wrong. :-)

In [ ]:
%%time
device = "cuda"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Format our fine-tuning data

We'll match the prompt setup we used above.

In [ ]:
def generate_prompt(data_point):
#     return f"""
#             {data_point["Question"]}. 
#             Answer as briefly as possible: {data_point["Answer"]}
#             """.strip()
    return f"""You are an AI assistant for a software project called {data_point["Repo"]}. You are trained on all the {content_type} that makes up this project.
    The docs for the project is located at {data_point["Repo Url"]}.
    You are given a repository which might contain several modules and each module will contain a set of files.
    Look at the source code in the repository and you have to generate content for the section of a README.md file following the heading given below. If you use any hyperlinks, they should link back to the github repository shared with you.
    You should only use hyperlinks that are explicitly listed in the context. Do NOT make up a hyperlink that is not listed.

    Assume the reader is a smart developer but is not deeply familiar with {data_point["Repo"]}.
    Assume the reader does not know anything about how the project is structured or which folders/files do what and what functions are written in which files and what these functions do.
    If you don't know how to fill up the readme.md file in one of its sections, leave that part blank. Don't try to make up any content.
    Do not include information that is not directly relevant to repository, even though the names of the functions might be common or is frequently used in several other places.
    Keep your response between 100 and 300 words. DO NOT RETURN MORE THAN 300 WORDS. Provide the answer in correct markdown format.

    Question: {data_point["Question"]}
    Context:
    {data_point["Context"]}

    Answer in Markdown:
    {data_point["Answer"]}
    """

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = Dataset.from_pandas(df)
data = data.shuffle().map(generate_and_tokenize_prompt)

## Train!

Now, we'll use our data to update our model. Using the Huggingface `transformers` library, let's set up our training loop and then run it. Note that we are ONLY making one pass on all this data.

In [ ]:
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA']="1"

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    output_dir="outputs",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,
    report_to="none"
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

## Loading and using the model later

Now, we'll save the PEFT fine-tuned model, then load it and use it to generate some more answers.

In [ ]:
model.save_pretrained("outputs/trained-model")
PEFT_MODEL = "outputs/trained-model"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 512
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.repetition_penalty = 1.1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
project_name = df["Repo"].values[10820]
repository_url = df["Repo Url"].values[10820]
target_audience = "smart developer"
question = df["Question"].values[10820]
context = df["Context"].values[10820]
content_type = "docs"
prompt = f"""You are an AI assistant for a software project called {project_name}. You are trained on all the {content_type} that makes up this project.
    The {content_type} for the project is located at {repository_url}.
    You are given a repository which might contain several modules and each module will contain a set of files.
    Look at the source code in the repository and you have to generate content for the section of a README.md file following the heading given below. If you use any hyperlinks, they should link back to the github repository shared with you.
    You should only use hyperlinks that are explicitly listed in the context. Do NOT make up a hyperlink that is not listed.

    Assume the reader is a {target_audience} but is not deeply familiar with {project_name}.
    Assume the reader does not know anything about how the project is structured or which folders/files do what and what functions are written in which files and what these functions do.
    If you don't know how to fill up the readme.md file in one of its sections, leave that part blank. Don't try to make up any content.
    Do not include information that is not directly relevant to repository, even though the names of the functions might be common or is frequently used in several other places.
    Keep your response between 100 and 300 words. DO NOT RETURN MORE THAN 300 WORDS. Provide the answer in correct markdown format.

    Question: {question}
    Context:
    {context}

    Answer in Markdown:"""
    
device = "cuda"
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))